In [ ]:
#!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c1226f07c5fa2593f5004c349029d06d9eac1623e6f572bc1a52cf4ab5030286
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [32]:
import os
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import min, max,min as min_, max as max_, to_date, year, concat_ws,substring, col, concat, lit
from pyspark.ml.feature import Bucketizer
from itertools import combinations
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSV").config("spark.sql.legacy.timeParserPolicy", "CORRECTED").getOrCreate()

current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Define the file name and its path relative to the current directory
file_name = "playstore.csv"

# Create the complete file path using os.path.join()
file_path = os.path.join(current_directory,file_name)
print("File Path:", file_path)
print(os.path.exists(file_path))
df = spark.read.csv(file_path, header=True, inferSchema=True)

Current Directory: C:\Users\Manish\Desktop\Segwise_Data_Engineer
File Path: C:\Users\Manish\Desktop\Segwise_Data_Engineer\playstore.csv
True


In [33]:
df = df.limit(2000)
columns = ["appId", "developerId", "free", "genre", "minInstalls", "price", "ratings", "adSupported", "containsAds", "reviews", 'releasedDayYear', 'sale', 'score', 'updated','dateUpdated', 'ParseReleasedDayYear']
df = df.select(*columns)

In [34]:
# Extracting the year from the 'releasedDayYear' column
df = df.withColumn("releasedYear", substring(col("releasedDayYear"), -4, 4).cast("int"))
df = df.filter((col("releasedYear") >= 1990) & (col("releasedYear") <= 2023))

In [35]:
# Filter rows with null values in any column
df = df.na.drop()
numeric_columns = ["free", "minInstalls", "releasedYear","price", "ratings", "adSupported", "containsAds", "reviews", 'sale', 'score', 'updated', 'ParseReleasedDayYear']
filter_condition = reduce(
    lambda acc, x: acc & col(x).rlike('^[0-9]*$'),
    numeric_columns[1:],
    col(numeric_columns[0]).rlike('^[0-9]*$')
)

# Filter rows where specified columns do not contain numeric values
df = df.filter(~filter_condition)

In [36]:
df = df.withColumn("minInstalls", df["minInstalls"].cast("int"))
df = df.withColumn("releasedYear", df["releasedYear"].cast("int"))
df = df.withColumn("price", df["price"].cast("float"))

In [37]:
# Get the minimum and maximum values for numerical columns
min_max_values = df.agg(
    min_("releasedYear").alias("min_released_year"),
    max_("releasedYear").alias("max_released_year"),
    min_("price").alias("min_price"),
    max_("price").alias("max_price"),
    min_("minInstalls").alias("min_installs"),
    max_("minInstalls").alias("max_installs")
).collect()[0]

In [38]:
# Extract min and max values for each column
min_released_year = min_max_values['min_released_year']
max_released_year = min_max_values['max_released_year']
min_price = min_max_values['min_price']
max_price = min_max_values['max_price']
min_installs = min_max_values['min_installs']
max_installs = min_max_values['max_installs']

year_interval = 2
# Create ranges for releasedYear with 2-year intervals
year_ranges = [x for x in range(min_released_year, max_released_year + 3, year_interval)]
print(year_ranges)
# Bucketizing 'releasedYear' column
bucketizer_year = Bucketizer(splits=year_ranges, inputCol="releasedYear", outputCol="year_bucket")
df = df.filter(~col("releasedYear").isNull())
df = bucketizer_year.transform(df)

[2013, 2015, 2017, 2019, 2021, 2023]


In [39]:
# Create ranges for installs with interval 100000
install_interval = 100000
install_ranges = [x for x in range(min_installs, max_installs + 1,install_interval)]
print(install_ranges)
# Bucketizing 'minInstalls' column
bucketizer_installs = Bucketizer(splits=install_ranges, inputCol="minInstalls", outputCol="install_bucket")
df = df.filter(~col("minInstalls").isNull())
df = bucketizer_installs.transform(df)

[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 2600000, 2700000, 2800000, 2900000, 3000000, 3100000, 3200000, 3300000, 3400000, 3500000, 3600000, 3700000, 3800000, 3900000, 4000000, 4100000, 4200000, 4300000, 4400000, 4500000, 4600000, 4700000, 4800000, 4900000, 5000000]


In [40]:
# Create ranges for price with interval 5
price_interval = 5
price_ranges = [x  for x in range(int(min_price * 2), int(max_price * 2) + 1,price_interval)]
print(price_ranges)
# Bucketizing 'price' column
bucketizer_price = Bucketizer(splits=price_ranges, inputCol="price", outputCol="price_bucket")
df = df.filter(~col("price").isNull())
df = bucketizer_price.transform(df)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495, 500, 505, 510, 515, 520, 525, 530, 535, 540, 545, 550, 555, 560, 565, 570, 575, 580, 585, 590, 595, 600, 605, 610, 615, 620, 625, 630, 635, 640, 645, 650, 655, 660, 665, 670, 675, 680, 685, 690, 695, 700, 705]


In [41]:
df = df.withColumn("year_bucket_range",concat(lit("["), (col("year_bucket")*year_interval+min_released_year).cast("int"), lit("-"),(col("year_bucket")*year_interval+min_released_year).cast("int")+year_interval-1, lit("]")))
df = df.withColumn("price_bucket_range",concat(lit("["), (col("price_bucket")*price_interval+min_price).cast("int"), lit("-"),(col("price_bucket")*price_interval+min_price).cast("int")+price_interval, lit("]")))
df = df.withColumn("install_bucket_range",concat(lit("["), (col("install_bucket")*install_interval+min_installs).cast("int"), lit("-"),(col("install_bucket")*install_interval+min_installs).cast("int")+install_interval, lit("]")))
df.show()

+--------------------+--------------------+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+------------+-------------------+--------------------+------------+-----------+--------------+------------+-----------------+------------------+--------------------+
|               appId|         developerId|free|       genre|minInstalls|price|ratings|adSupported|containsAds|reviews|releasedDayYear|sale|score|     updated|        dateUpdated|ParseReleasedDayYear|releasedYear|year_bucket|install_bucket|price_bucket|year_bucket_range|price_bucket_range|install_bucket_range|
+--------------------+--------------------+----+------------+-----------+-----+-------+-----------+-----------+-------+---------------+----+-----+------------+-------------------+--------------------+------------+-----------+--------------+------------+-----------------+------------------+--------------------+
|com.cyou.cma.clau...| 7398289236941441420|   1|Art & Design|   

In [42]:
def write_csv(dft, outputFilename):
    file_path = os.path.join(current_directory,outputFilename)
    dft.write.csv(file_path, header=True, mode='overwrite')

In [43]:
selected_columns = ["developerId","free", "genre", "price", "ratings", "releasedYear","adSupported","containsAds","reviews","score","updated","year_bucket_range","install_bucket_range", "price_bucket_range"]
# Define the range of permutation sizes (3 in this case) 
min_comb_size = 3
max_comb_size = 3

# Create an empty DataFrame to hold aggregated results
schema = StructType([
    StructField("property", StringType(), True),
    StructField("count", IntegerType(), True)
])

# Create an empty DataFrame with the defined schema
result_df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

# Loop through different combinations sizes
for comb_size in range(min_comb_size, max_comb_size + 1):
    # Generate combinations of selected_columns for the current perm_size without duplicates
    combinations_without_duplicates = list(combinations(selected_columns, comb_size))

    # Iterate through each combination for the current perm_size
    for combination in combinations_without_duplicates:
        combination_list = list(combination)
        print(combination_list)

        properties = [concat(concat_ws("=", lit(column), col(column)),lit(";")).alias(column) for column in combination_list]
        combined_properties = concat(*properties).alias("property")
        # Aggregate count for each unique combination of properties
        aggregation = df.groupBy(*combination_list).count().withColumnRenamed("count", "count")
        aggregation.show(truncate=False)
        # Union the aggregation results with the result_df
        result_df = result_df.union(aggregation.select(combined_properties, col("count")))
        result_df = result_df.where(col("count")>20)

['developerId', 'free', 'genre']
+------------------------------+----+------------+-----+
|developerId                   |free|genre       |count|
+------------------------------+----+------------+-----+
|7398289236941441420           |1   |Art & Design|1    |
|5762674363677973291           |1   |Art & Design|1    |
|NAVEENKUMAR                   |1   |Art & Design|1    |
|360+Fractal                   |1   |Art & Design|1    |
|MUSMon.com                    |0   |Art & Design|1    |
|Wiedmann+Media+AG             |0   |Art & Design|1    |
|Fashion+Club                  |1   |Art & Design|1    |
|Make+in+India+Apps            |1   |Art & Design|1    |
|ShinChven+Zhang               |1   |Art & Design|1    |
|DrawAPP                       |1   |Art & Design|1    |
|%2BHOME+by+Ateam+Entertainment|1   |Art & Design|8    |
|Inforotor                     |1   |Art & Design|3    |
|7832284125212021063           |1   |Art & Design|1    |
|App+Wala                      |1   |Art & Design|1    

+------------------------------+----+-------+-----+
|developerId                   |free|reviews|count|
+------------------------------+----+-------+-----+
|7398289236941441420           |1   |152    |1    |
|5762674363677973291           |1   |62     |1    |
|NAVEENKUMAR                   |1   |59     |1    |
|360+Fractal                   |1   |58     |1    |
|MUSMon.com                    |0   |5      |1    |
|Wiedmann+Media+AG             |0   |0      |1    |
|Fashion+Club                  |1   |0      |1    |
|Make+in+India+Apps            |1   |0      |1    |
|ShinChven+Zhang               |1   |78     |1    |
|DrawAPP                       |1   |360    |1    |
|%2BHOME+by+Ateam+Entertainment|1   |9      |1    |
|%2BHOME+by+Ateam+Entertainment|1   |10     |1    |
|Inforotor                     |1   |0      |3    |
|7832284125212021063           |1   |0      |1    |
|App+Wala                      |1   |9      |1    |
|TRENDYBOYS                    |1   |0      |1    |
|83481357962

+------------------------------+------------+-----+-----+
|developerId                   |genre       |price|count|
+------------------------------+------------+-----+-----+
|7398289236941441420           |Art & Design|0.0  |1    |
|5762674363677973291           |Art & Design|0.0  |1    |
|NAVEENKUMAR                   |Art & Design|0.0  |1    |
|360+Fractal                   |Art & Design|0.0  |1    |
|MUSMon.com                    |Art & Design|2.61 |1    |
|Wiedmann+Media+AG             |Art & Design|9.9  |1    |
|Fashion+Club                  |Art & Design|0.0  |1    |
|Make+in+India+Apps            |Art & Design|0.0  |1    |
|ShinChven+Zhang               |Art & Design|0.0  |1    |
|DrawAPP                       |Art & Design|0.0  |1    |
|%2BHOME+by+Ateam+Entertainment|Art & Design|0.0  |8    |
|Inforotor                     |Art & Design|0.0  |3    |
|7832284125212021063           |Art & Design|0.0  |1    |
|App+Wala                      |Art & Design|0.0  |1    |
|TRENDYBOYS   

+------------------------------+------------+-----+-----+
|developerId                   |genre       |score|count|
+------------------------------+------------+-----+-----+
|7398289236941441420           |Art & Design|4.75 |1    |
|5762674363677973291           |Art & Design|4.52 |1    |
|NAVEENKUMAR                   |Art & Design|3.61 |1    |
|360+Fractal                   |Art & Design|4.5  |1    |
|MUSMon.com                    |Art & Design|4.0  |1    |
|Wiedmann+Media+AG             |Art & Design|0.0  |1    |
|Fashion+Club                  |Art & Design|0.0  |1    |
|Make+in+India+Apps            |Art & Design|0.0  |1    |
|ShinChven+Zhang               |Art & Design|3.94 |1    |
|DrawAPP                       |Art & Design|3.9  |1    |
|%2BHOME+by+Ateam+Entertainment|Art & Design|4.68 |1    |
|%2BHOME+by+Ateam+Entertainment|Art & Design|4.83 |1    |
|Inforotor                     |Art & Design|0.0  |3    |
|7832284125212021063           |Art & Design|0.0  |1    |
|App+Wala     

+------------------------------+-----+-------+-----+
|developerId                   |price|ratings|count|
+------------------------------+-----+-------+-----+
|7398289236941441420           |0.0  |30650  |1    |
|5762674363677973291           |0.0  |15150  |1    |
|NAVEENKUMAR                   |0.0  |3312   |1    |
|360+Fractal                   |0.0  |915    |1    |
|MUSMon.com                    |2.61 |38     |1    |
|Wiedmann+Media+AG             |9.9  |0      |1    |
|Fashion+Club                  |0.0  |0      |1    |
|Make+in+India+Apps            |0.0  |0      |1    |
|ShinChven+Zhang               |0.0  |629    |1    |
|DrawAPP                       |0.0  |6864   |1    |
|%2BHOME+by+Ateam+Entertainment|0.0  |437    |1    |
|%2BHOME+by+Ateam+Entertainment|0.0  |177    |1    |
|Inforotor                     |0.0  |0      |3    |
|7832284125212021063           |0.0  |0      |1    |
|App+Wala                      |0.0  |140    |1    |
|TRENDYBOYS                    |0.0  |0      |

...

['containsAds', 'reviews', 'price_bucket_range']
+-----------+-------+------------------+-----+
|containsAds|reviews|price_bucket_range|count|
+-----------+-------+------------------+-----+
|1          |152    |[0-5]             |1    |
|1          |62     |[0-5]             |1    |
|1          |59     |[0-5]             |1    |
|1          |58     |[0-5]             |1    |
|0          |5      |[0-5]             |2    |
|0          |0      |[5-10]            |6    |
|1          |0      |[0-5]             |1357 |
|1          |78     |[0-5]             |1    |
|1          |360    |[0-5]             |1    |
|1          |9      |[0-5]             |3    |
|1          |10     |[0-5]             |3    |
|0          |0      |[0-5]             |440  |
|1          |8      |[0-5]             |7    |
|1          |2      |[0-5]             |18   |
|1          |18     |[0-5]             |2    |
|1          |49     |[0-5]             |1    |
|1          |7      |[0-5]             |10   |
|1         

['containsAds', 'updated', 'price_bucket_range']
+-----------+------------+------------------+-----+
|containsAds|updated     |price_bucket_range|count|
+-----------+------------+------------------+-----+
|1          |1555921116.0|[0-5]             |1    |
|1          |1555921666.0|[0-5]             |1    |
|1          |1640242939.0|[0-5]             |1    |
|1          |1537435772.0|[0-5]             |1    |
|0          |1403192042.0|[0-5]             |1    |
|0          |1664662424.0|[5-10]            |1    |
|1          |1465326164.0|[0-5]             |1    |
|1          |1481880380.0|[0-5]             |1    |
|1          |1566435903.0|[0-5]             |1    |
|1          |1660664062.0|[0-5]             |1    |
|1          |1473650319.0|[0-5]             |1    |
|1          |1475577037.0|[0-5]             |1    |
|0          |1575451764.0|[0-5]             |1    |
|0          |1575445356.0|[0-5]             |1    |
|1          |1564160075.0|[0-5]             |1    |
|1          |16

['reviews', 'score', 'price_bucket_range']
+-------+-----+------------------+-----+
|reviews|score|price_bucket_range|count|
+-------+-----+------------------+-----+
|152    |4.75 |[0-5]             |1    |
|62     |4.52 |[0-5]             |1    |
|59     |3.61 |[0-5]             |1    |
|58     |4.5  |[0-5]             |1    |
|5      |4.0  |[0-5]             |1    |
|0      |0.0  |[5-10]            |6    |
|0      |0.0  |[0-5]             |1791 |
|78     |3.94 |[0-5]             |1    |
|360    |3.9  |[0-5]             |1    |
|9      |4.68 |[0-5]             |1    |
|10     |4.83 |[0-5]             |1    |
|9      |4.13 |[0-5]             |1    |
|8      |3.59 |[0-5]             |1    |
|2      |4.17 |[0-5]             |1    |
|18     |4.43 |[0-5]             |1    |
|8      |2.3  |[0-5]             |1    |
|49     |1.72 |[0-5]             |1    |
|7      |4.5  |[0-5]             |1    |
|12     |4.06 |[0-5]             |1    |
|13     |4.57 |[0-5]             |1    |
+-------+-----

['score', 'updated', 'year_bucket_range']
+-----+------------+-----------------+-----+
|score|updated     |year_bucket_range|count|
+-----+------------+-----------------+-----+
|4.75 |1555921116.0|[2013-2014]      |1    |
|4.52 |1555921666.0|[2013-2014]      |1    |
|3.61 |1640242939.0|[2013-2014]      |1    |
|4.5  |1537435772.0|[2013-2014]      |1    |
|4.0  |1403192042.0|[2013-2014]      |1    |
|0.0  |1664662424.0|[2015-2016]      |1    |
|0.0  |1465326164.0|[2015-2016]      |1    |
|0.0  |1481880380.0|[2015-2016]      |1    |
|3.94 |1566435903.0|[2015-2016]      |1    |
|3.9  |1660664062.0|[2015-2016]      |1    |
|4.68 |1473650319.0|[2015-2016]      |1    |
|4.83 |1475577037.0|[2015-2016]      |1    |
|0.0  |1575451764.0|[2015-2016]      |1    |
|0.0  |1575445356.0|[2015-2016]      |1    |
|0.0  |1564160075.0|[2015-2016]      |1    |
|4.13 |1610018300.0|[2017-2018]      |1    |
|0.0  |1547145561.0|[2017-2018]      |1    |
|0.0  |1476767388.0|[2015-2016]      |1    |
|0.0  |162991

['updated', 'year_bucket_range', 'price_bucket_range']
+------------+-----------------+------------------+-----+
|updated     |year_bucket_range|price_bucket_range|count|
+------------+-----------------+------------------+-----+
|1555921116.0|[2013-2014]      |[0-5]             |1    |
|1555921666.0|[2013-2014]      |[0-5]             |1    |
|1640242939.0|[2013-2014]      |[0-5]             |1    |
|1537435772.0|[2013-2014]      |[0-5]             |1    |
|1403192042.0|[2013-2014]      |[0-5]             |1    |
|1664662424.0|[2015-2016]      |[5-10]            |1    |
|1465326164.0|[2015-2016]      |[0-5]             |1    |
|1481880380.0|[2015-2016]      |[0-5]             |1    |
|1566435903.0|[2015-2016]      |[0-5]             |1    |
|1660664062.0|[2015-2016]      |[0-5]             |1    |
|1473650319.0|[2015-2016]      |[0-5]             |1    |
|1475577037.0|[2015-2016]      |[0-5]             |1    |
|1575451764.0|[2015-2016]      |[0-5]             |1    |
|1575445356.0|[20

In [44]:
# Write the aggregated results of all permutations to a single CSV file
if result_df.count() > 0:
    try:
        write_csv(result_df.coalesce(1),"output_all_insights_sp.csv")
    except Exception as e:
        print("Trying to write dataframe in csv after conversion to pandas.")
        final_df = result_df.toPandas()
        with open("output_all_insights.csv", mode='w') as file:
            final_df.to_csv(file, index=False)
        
else:
    print("No data to write. DataFrame is empty.")
spark.stop()

Trying to write dataframe in csv after conversion to pandas.
